In [29]:
import cv2
from PIL import Image
import numpy as np
import tensorflow as tf
import functools

In [23]:
test_image = cv2.imread('/Users/johnmorin/KaylaTek/Data/Flight_5_with_ground_truth/frame_000000.tif')
test_image2 = cv2.imread('/Users/johnmorin/KaylaTek/Data/Flight_5_with_ground_truth/frame_000001.tif')

In [3]:
jpg_im_loc = '/Users/johnmorin/Documents/GitHub/Obj-Det-Tutorial/TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10/images/train/cam_image1.jpg'

In [4]:
jpg_image_cv2 = cv2.imread(jpg_im_loc)
jpg_image_PIL = Image.open(jpg_im_loc)

In [5]:
with tf.gfile.GFile(jpg_im_loc, 'rb') as fid:
        encoded_jpg = fid.read()

In [6]:
tf_image = tf.image.convert_image_dtype(
    image=test_image,
    dtype='uint8',
    saturate=False,
    name=None
)

In [24]:
tf_image2 = tf.image.convert_image_dtype(
    image=test_image2,
    dtype='uint8',
    saturate=False,
    name=None
)

In [11]:
jpg_tf_image = tf.image.convert_image_dtype(
    image=jpg_image_cv2,
    dtype='uint8',
    saturate=False,
    name=None
)

In [10]:
def draw_bounding_boxes_on_image_tensors(images,
                                        boxes,
                                        classes,
                                        scores,
                                        category_index,
                                        original_image_spatial_shape=None,
                                        true_image_shape=None,
                                        instance_masks=None,
                                        keypoints=None,
                                        max_boxes_to_draw=20,
                                        min_score_thresh=0.2,
                                        use_normalized_coordinates=True):
    """Draws bounding boxes, masks, and keypoints on batch of image tensors.
    Args:
        images: A 4D uint8 image tensor of shape [N, H, W, C]. If C > 3, additional
            channels will be ignored. If C = 1, then we convert the images to RGB
            images.
        boxes: [N, max_detections, 4] float32 tensor of detection boxes.
        classes: [N, max_detections] int tensor of detection classes. Note that
            classes are 1-indexed.
        scores: [N, max_detections] float32 tensor of detection scores.
        category_index: a dict that maps integer ids to category dicts. e.g.
            {1: {1: 'dog'}, 2: {2: 'cat'}, ...}
        original_image_spatial_shape: [N, 2] tensor containing the spatial size of
            the original image.
        true_image_shape: [N, 3] tensor containing the spatial size of unpadded
            original_image.
        instance_masks: A 4D uint8 tensor of shape [N, max_detection, H, W] with
            instance masks.
        keypoints: A 4D float32 tensor of shape [N, max_detection, num_keypoints, 2]
            with keypoints.
        max_boxes_to_draw: Maximum number of boxes to draw on an image. Default 20.
        min_score_thresh: Minimum score threshold for visualization. Default 0.2.
        use_normalized_coordinates: Whether to assume boxes and kepoints are in
            normalized coordinates (as opposed to absolute coordiantes).
            Default is True.
    Returns:
        4D image tensor of type uint8, with boxes drawn on top.
    """
    # Additional channels are being ignored.
    if images.shape[3] > 3:
        images = images[:, :, :, 0:3]
    elif images.shape[3] == 1:
        images = tf.image.grayscale_to_rgb(images)
    visualization_keyword_args = {
            'use_normalized_coordinates': use_normalized_coordinates,
            'max_boxes_to_draw': max_boxes_to_draw,
            'min_score_thresh': min_score_thresh,
            'agnostic_mode': False,
            'line_thickness': 4
    }
    if true_image_shape is None:
        true_shapes = tf.constant(-1, shape=[images.shape.as_list()[0], 3])
    else:
        true_shapes = true_image_shape
    if original_image_spatial_shape is None:
        original_shapes = tf.constant(-1, shape=[images.shape.as_list()[0], 2])
    else:
        original_shapes = original_image_spatial_shape

    if instance_masks is not None and keypoints is None:
        visualize_boxes_fn = functools.partial(
                _visualize_boxes_and_masks,
                category_index=category_index,
                **visualization_keyword_args)
        elems = [
                true_shapes, original_shapes, images, boxes, classes, scores,
                instance_masks
        ]
    elif instance_masks is None and keypoints is not None:
        visualize_boxes_fn = functools.partial(
                _visualize_boxes_and_keypoints,
                category_index=category_index,
                **visualization_keyword_args)
        elems = [
                true_shapes, original_shapes, images, boxes, classes, scores, keypoints
        ]
    elif instance_masks is not None and keypoints is not None:
        visualize_boxes_fn = functools.partial(
                _visualize_boxes_and_masks_and_keypoints,
                category_index=category_index,
                **visualization_keyword_args)
        elems = [
                true_shapes, original_shapes, images, boxes, classes, scores,
                instance_masks, keypoints
        ]
    else:
        visualize_boxes_fn = functools.partial(
                _visualize_boxes,
                category_index=category_index,
                **visualization_keyword_args)
        elems = [
                true_shapes, original_shapes, images, boxes, classes, scores
        ]

    def draw_boxes(image_and_detections):
        """Draws boxes on image."""
        true_shape = image_and_detections[0]
        original_shape = image_and_detections[1]
        if true_image_shape is not None:
            image = shape_utils.pad_or_clip_nd(image_and_detections[2],
                                                                                 [true_shape[0], true_shape[1], 3])
        if original_image_spatial_shape is not None:
            image_and_detections[2] = _resize_original_image(image, original_shape)

        image_with_boxes = tf.py_func(visualize_boxes_fn, image_and_detections[2:],
                                                                    tf.uint8)
        return image_with_boxes

    images = tf.map_fn(draw_boxes, elems, dtype=tf.uint8, back_prop=False)
    return images

In [31]:
def _visualize_boxes(image, boxes, classes, scores, category_index, **kwargs):
    return visualize_boxes_and_labels_on_image_array(
      image, boxes, classes, scores, category_index=category_index, **kwargs)

In [18]:
tf_image.shape[2]

Dimension(3)

In [26]:
stacked_image = tf.stack(
    [tf_image, tf_image2],
    axis=0,
)

In [27]:
stacked_image.shape[3]

Dimension(3)

In [35]:
print(tf.summary.image(
    name='test',
    tensor=tf_image
))

Tensor("test_2:0", shape=(), dtype=string)


In [32]:
draw_bounding_boxes_on_image_tensors(images=stacked_image,
                                     boxes=[[(1074/1280), (438/1024), (1096/1280), (455/1024)], 1, 4], 
                                     classes=['target', 1],
                                     scores=[1, 1],
                                     category_index={1: {1: 'target'}}
                                    )

ValueError: slice index 0 of dimension 0 out of bounds. for 'map/TensorArrayUnstack_3/strided_slice' (op: 'StridedSlice') with input shapes: [0], [1], [1], [1] and with computed input tensors: input[1] = <0>, input[2] = <1>, input[3] = <1>.

In [ ]:
def draw_bounding_box_on_image_array(image,
                                     ymin,
                                     xmin,
                                     ymax,
                                     xmax,
                                     color='red',
                                     thickness=4,
                                     display_str_list=(),
                                     use_normalized_coordinates=True):
    """Adds a bounding box to an image (numpy array).
    Bounding box coordinates can be specified in either absolute (pixel) or
    normalized coordinates by setting the use_normalized_coordinates argument.
    Args:
        image: a numpy array with shape [height, width, 3].
        ymin: ymin of bounding box.
        xmin: xmin of bounding box.
        ymax: ymax of bounding box.
        xmax: xmax of bounding box.
        color: color to draw bounding box. Default is red.
        thickness: line thickness. Default value is 4.
        display_str_list: list of strings to display in box
                      (each to be shown on its own line).
        use_normalized_coordinates: If True (default), treat coordinates
        ymin, xmin, ymax, xmax as relative to the image.  Otherwise treat
        coordinates as absolute.
    """
    image_pil = Image.fromarray(np.uint8(image)).convert('RGB')
    draw_bounding_box_on_image(image_pil, ymin, xmin, ymax, xmax, color,
                             thickness, display_str_list,
                             use_normalized_coordinates)
    np.copyto(image, np.array(image_pil))

In [ ]:
tf.image.draw_bounding_boxes(
    [tf_image],
    (1074/1280), (438/1024), (1096/1280), (455/1024),
    name='Target'
)

In [61]:
cv2_tf_image = tf.image.decode_jpeg(
    jpg_image_cv2.tostring(),
    channels=0,
    ratio=1,
    fancy_upscaling=True,
    try_recover_truncated=False,
    acceptable_fraction=1,
    dct_method='',
    name=None
)

In [60]:
cv2_tf_image2 = tf.image.decode_image(
    contents=jpg_image_cv2.tostring(),
    channels=None,
)

In [63]:
print(type(cv2_tf_image))
print(cv2_tf_image.shape)
print(type(cv2_tf_image2))
print(cv2_tf_image2.shape)

<class 'tensorflow.python.framework.ops.Tensor'>
(?, ?, ?)
<class 'tensorflow.python.framework.ops.Tensor'>
<unknown>


In [80]:
PIL_tf_image = tf.image.decode_jpeg(
    PIL_jpg,
    channels=0,
    ratio=1,
    fancy_upscaling=True,
    try_recover_truncated=False,
    acceptable_fraction=1,
    dct_method='',
    name=None
)

In [81]:
PIL_tf_image2 = tf.image.decode_image(
    contents=PIL_jpg,
    channels=None,
)

In [75]:
tf.image.extract_jpeg_shape(
    PIL_jpg,
    output_type=tf.int32,
    name=None
)

<tf.Tensor 'ExtractJpegShape:0' shape=(3,) dtype=int32>

In [76]:
tf.image.extract_jpeg_shape(
    jpg_image_cv2.tostring(),
    output_type=tf.int32,
    name=None
)

<tf.Tensor 'ExtractJpegShape_1:0' shape=(3,) dtype=int32>

In [85]:
tfgfile_tf_image = tf.image.decode_image(
    contents=encoded_jpg,
    channels=None,
)

In [86]:
print(type(tfgfile_tf_image))
print(tfgfile_tf_image.shape)

<class 'tensorflow.python.framework.ops.Tensor'>
<unknown>


In [90]:
print(tf.image.is_jpeg(
    jpg_image_cv2.tostring(),
    name=None
))

Tensor("is_jpeg/Equal:0", shape=(), dtype=bool)


In [91]:
print(tf.image.is_jpeg(
    encoded_jpg,
    name=None
))

Tensor("is_jpeg_1/Equal:0", shape=(), dtype=bool)


In [93]:
if '\xff\xd8\xff\xe0' in str(jpg_image_cv2.tostring()):
    print('True')

In [103]:
str(jpg_image_cv2.tostring())[:20]

"b'2L\\\\<VdVlw}\\x90\\x9"

In [96]:
jpg_image_cv2 = cv2.imread('/Users/johnmorin/Documents/GitHub/Obj-Det-Tutorial/TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10/images/train/IMG_2385.JPG')

In [102]:
str(jpg_image_cv2.tostring())[:20]

"b'2L\\\\<VdVlw}\\x90\\x9"

In [101]:
test3 = cv2.imread('/Users/johnmorin/Downloads/file.jpeg')
print(str(test3.tostring())[:20])

b'\x96\xa5\xb5\x93\x


In [100]:
test4 = cv2.imread('/Users/johnmorin/KaylaTek/tank_pic.jpg')
print(str(test4.tostring())[:20])

b'\xdd\xdd\xcd\xdd\x


In [104]:
with tf.gfile.GFile('/Users/johnmorin/KaylaTek/tank_pic.jpg', 'rb') as fid:
    tank_jpg = fid.read()
str(tank_jpg)[:20]

"b'\\xff\\xd8\\xff\\xe0\\x"

In [114]:
tf.image.extract_jpeg_shape(
    tank_jpg,
    output_type=tf.int32,
    name=None
)

<tf.Tensor 'ExtractJpegShape_2:0' shape=(3,) dtype=int32>

In [117]:
tf.image.decode_image(
    contents=tank_jpg,
    channels=None,
)

<tf.Tensor 'decode_image_12/cond_jpeg/Merge:0' shape=<unknown> dtype=uint8>

In [119]:
omg = tf.image.decode_jpeg(
    tank_jpg,
    channels=0,
    ratio=1,
    fancy_upscaling=True,
    try_recover_truncated=False,
    acceptable_fraction=1,
    dct_method='',
    name=None
)

In [120]:
omg.shape

TensorShape([Dimension(None), Dimension(None), Dimension(None)])

In [112]:
with tf.gfile.GFile('/Users/johnmorin/Downloads/file.jpeg', 'rb') as fid:
    file_jpg = fid.read()
str(file_jpg)[:20]

"b'\\xff\\xd8\\xff\\xe0\\x"

In [113]:
with tf.gfile.GFile('/Users/johnmorin/Documents/GitHub/Obj-Det-Tutorial/TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10/images/train/IMG_2385.JPG', 'rb') as fid:
    obj_jpg = fid.read()
str(obj_jpg)[:20]

"b'\\xff\\xd8\\xff\\xe0\\x"

In [108]:
tankbool = tf.image.is_jpeg(
    tank_jpg,
    name=None
)

In [111]:
#a = tf.placeholder(tankbool)  #placeholder for a single boolean value
b = tf.cond(
        tf.equal(tankbool, tf.constant(True)), 
        lambda: tf.constant(10), 
        lambda: tf.constant(0))
sess = tf.InteractiveSession()
res = sess.run(b, feed_dict = {tankbool: True})
sess.close()
print(res)

10


/Users/johnmorin/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py:1714: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [121]:
tiff = '/Users/johnmorin/KaylaTek/Data/Flight_5_with_ground_truth/frame_000000.tif'

In [123]:
with tf.gfile.GFile(tiff, 'rb') as fid:
    tiff_obj = fid.read()
str(tiff_obj)[:20]

"b'II*\\x00\\x08\\x00\\x0"